In [53]:
# 🧩 Schritt 1: Imports und Setup
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example
import json
import random
from pathlib import Path
from spacy.util import minibatch
from spacy.util import compounding
!python -m spacy download de_core_news_md
!pip install spacy-lookups-data



# 📁 Schritt 2: Funktion zum Laden von Daten aus JSON
def load_data_from_json(path):
    with open(path, "r", encoding="utf-8") as f:
        raw_data = json.load(f)
    if isinstance(raw_data, dict):
        raw_data = [raw_data]

    TRAIN_DATA = []
    for entry in raw_data:
        text = entry["text"]
        entities = [(label["start"], label["end"], label["label"]) for label in entry["labels"]]
        TRAIN_DATA.append((text, {"entities": entities}))
    return TRAIN_DATA

# 🔄 Lade Trainings- und Dev-Daten separat
train_data = load_data_from_json("../../../data/synthetic/synthetic_mails_option_b.json")
dev_data = load_data_from_json("../../../data/original/ground_truth_split/validation_norm.json")
print(f"📥 Trainingsbeispiele: {len(train_data)}, Dev-Beispiele: {len(dev_data)}")

# 🧠 Schritt 3: Lade spaCy-Basismodell
base_model = "de_core_news_md"
nlp = spacy.load(base_model)


# Stelle sicher, dass NER-Komponente existiert
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# Registriere alle Labels aus beiden Datensätzen
for dataset in (train_data, dev_data):
    for _, annotations in dataset:
        for start, end, label in annotations["entities"]:
            ner.add_label(label)

# 🚀 Schritt 4: Modell-Initialisierung mit allen Daten (nur für Labels!)
def get_examples():
    for text, ann in train_data + dev_data:
        yield Example.from_dict(nlp.make_doc(text), ann)

optimizer = nlp.resume_training()


# 🏋️ Schritt 5: Training (nur auf Trainingsdaten)
n_iter = 20
for i in range(n_iter):
    random.shuffle(train_data)
    losses = {}

    batches = minibatch(train_data, size=compounding(8.0, 64.0, 1.001))
    for batch in batches:
        examples = [Example.from_dict(nlp.make_doc(text), ann) for text, ann in batch]
        nlp.update(examples, drop=0.2, losses=losses)

    print(f"🔁 Iteration {i+1}/{n_iter}, Loss: {losses['ner']:.4f}")

# 💾 Schritt 6: Modell speichern
output_dir = Path("custom_spacy_model_synthetic_data")
output_dir.mkdir(exist_ok=True)
nlp.to_disk(output_dir)
print(f"\n✅ Modell gespeichert unter: {output_dir.resolve()}")

# 🔍 Schritt 7: Modell laden und auf dev_data testen
nlp = spacy.load(output_dir)

print("\n📊 Evaluation auf dev_data:")
for text, _ in random.sample(dev_data, min(5, len(dev_data))):  # max. 5 Beispiele
    doc = nlp2(text)
    print(f"\n> {text}")
    for ent in doc.ents:
        print(f"  - {ent.text} ({ent.label_})")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 31.4 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')
📥 Trainingsbeispiele: 14360, Dev-Beispiele: 25


/opt/anaconda3/envs/piranha-env/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Sehr geehrte Damen und Herren,
bitte entnehmen Sie..." with entities "[(117, 122, 'FIRMA'), (123, 128, 'VORNAME'), (129,...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda3/envs/piranha-env/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Seit 16.03.1972 wird mein Abschlag nicht berechnet..." with entities "[(5, 15, 'DATUM'), (161, 173, 'VERTRAGSNUMMER'), (...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/anaconda3/envs/piranha-env/lib/python3.10/site-packages/spacy/training/io

KeyboardInterrupt: 

In [50]:
# 🔍 Schritt 7: Modell laden
nlp2 = spacy.load("custom_spacy_model_doccano_labeling")

# 🧩 Schritt 7.1: EntityRuler hinzufügen
# ❌ Entferne ggf. vorher vorhandenen EntityRuler
if "entity_ruler" in nlp2.pipe_names:
    nlp2.remove_pipe("entity_ruler")

# ✅ EntityRuler nach dem NER einfügen, damit er bevorzugt wird
ruler = nlp2.add_pipe("entity_ruler", before="ner")
# Beispiel: Lade Muster
# Funktion zum Laden von Namen aus Datei
def load_names(path, label):
    with open(path, "r", encoding="utf-8") as f:
          names = [name.strip() for name in f if name.strip()]
          patterns = [{"label": label, "pattern": name} for name in names]
    return patterns, names

# Gazetteer laden
vornamen_patterns, vornamen_liste = load_names("Vornamen.txt", "VORNAME")
nachnamen_patterns, nachnamen_liste = load_names("Nachnamen.txt", "NACHNAME")
titel_patterns, titel_liste = load_names("Titel.txt", "TITEL")
wohnort_patterns, wohnort_liste = load_names("Orte.txt", "WOHNORT")
postleitzahl_patterns, postleitzahl_liste = load_names("Postleitzahlen.txt", "POSTLEITZAHL")
strasse_patterns = [
    {
        "label": "STRASSE",
        "pattern": [
            {"TEXT": {"REGEX": r".*(straße|gasse|allee|weg|platz|str.|grund)$"}},
            {"TEXT": {"REGEX": r"^\d+[a-zA-Z]?$"}}
        ]
    }
]
vertragsnummer_patterns = [
    {
        "label": "VERTRAGSNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["vertragsnummer", "vertragsnr.", "vnr", "vn"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{6,12}\.?$"}}

        ]
    }
]

kundennummer_patterns = [
    {
        "label": "KUNDENNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["kundennummer", "kundennr.", "kdnr", "kd"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{6,12}\.?$"}}
        ]
    }
]

zuordnungsnummer_patterns = [
    {
        "label": "ZUORDNUNGSNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["znr", "zuordnungsnummer"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{6,12}\.?$"}}
        ]
    }
]
iban_pattern = [
    {"label": "IBAN", "pattern": [{"TEXT": {"REGEX": r"^[A-Z]{2}[0-9]{2}[A-Z0-9]{11,30}$"}}]}
]

bic_pattern = [
    {"label": "BIC", "pattern": [{"TEXT": {"REGEX": r"^[A-Z]{6}[A-Z2-9][A-NP-Z0-9]([A-Z0-9]{3})?$"}}]}
]

zahlung_pattern = [
    {
        "label": "ZAHLUNG",
        "pattern": [
            {"TEXT": {"REGEX": r"^\d+[.,]?\d{0,2}$"}},
            {"TEXT": {"REGEX": r"^(€|euro|eur)$"}}
        ]
    }
]

zählerstand_patterns = [
    {
        "label": "ZÄHLERSTAND",
        "pattern": [
            {"LOWER": {"IN": ["zählerstand"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d+(\.\d+)?$"}}
        ]
    },
    {
        "label": "ZÄHLERSTAND",
        "pattern": [
            {"LOWER": {"IN": ["zählerstand"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"IN": [":"]}, "OP": "?"},
            {"TEXT": {"REGEX": r"^\d{1,5}([.,]\d{1,2})?$"}}
        ]
    }
]

zählernummer_patterns = [
    {
        "label": "ZÄHLERNUMMER",
        "pattern": [
            {"LOWER": {"IN": ["zählernummer"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^[A-Z0-9]{6,20}$"}}  # Groß- und Kleinbuchstaben + Ziffern erlaubt
        ]
    }
]

verbrauch_patterns = [
    {
        "label": "VERBRAUCH",
        "pattern": [
            {"LOWER": {"IN": ["verbrauch"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d+(\.\d+)?$"}},
            {"LOWER": {"IN": ["kwh", "m³", "kw"]}, "OP": "?"}
        ]
    }
]

verbrauch_patterns += [
    {
        "label": "VERBRAUCH",
        "pattern": [
            {"LOWER": {"IN": ["verbrauch"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d+(?:[.,]\d+)?(kwh|m³|kw)$"}}
        ]
    }
]


wlv_patterns = [
    {
        "label": "WLV",
        "pattern": [
            {"LOWER": {"IN": ["wlv"]}},
            {"IS_PUNCT": True, "OP": "*"},
            {"TEXT": {"REGEX": r"^\d{4,12}$"}}
        ]
    }
]

email_pattern = [
    {
        "label": "EMAIL",
        "pattern": [
            {"TEXT": {"REGEX": r"^[\w\.-]+@[\w\.-]+\.\w{2,}$"}}
        ]
    }
]

telefon_pattern = [
    {
        "label": "TELEFON",
        "pattern": [
            {"TEXT": {"REGEX": r"^(\+49|0)[\d\s/-]{7,}$"}}
        ]
    }
]

url_pattern = [
    {
        "label": "LINK",
        "pattern": [
            {"TEXT": {"REGEX": r"^https?://[\w\-\.]+\.\w{2,}(/[\w\-\.]*)*$"}}
        ]
    },
    {
        "label": "LINK",
        "pattern": [
            {"TEXT": {"REGEX": r"^www\.[\w\-\.]+\.\w{2,}(/[\w\-\.]*)*$"}}
        ]
    }
]

datum_pattern = [
    {
        "label": "DATUM",
        "pattern": [
            {"TEXT": {"REGEX": r"^(\d{1,2}[./-]){2}\d{2,4}$"}}  # z. B. 15.06.2024
        ]
    },
    {
        "label": "DATUM",
        "pattern": [
            {"TEXT": {"REGEX": r"^\d{4}-\d{2}-\d{2}$"}}  # z. B. 2024-06-15
        ]
    },
    {
        "label": "DATUM",
        "pattern": [
            {"TEXT": {"REGEX": r"^\d{1,2}$"}},  # z. B. 15
            {"LOWER": {"IN": [
                "januar", "jan", "februar", "feb", "märz", "maerz", "mrz", "april", "apr",
                "mai", "juni", "jun", "juli", "jul", "august", "aug", "september", "sep",
                "oktober", "okt", "november", "nov", "dezember", "dez"
            ]}},
            {"TEXT": {"REGEX": r"^\d{2,4}$"}, "OP": "?"}  # optional Jahr
        ]
    }
]





# EntityRuler erstellen und Muster hinzufügen

ruler.add_patterns(zahlung_pattern + url_pattern + iban_pattern + bic_pattern + zahlung_pattern + zählerstand_patterns + email_pattern + telefon_pattern)  # 👈 Muster hinzufügen!


# 💾 (Optional) Modell MIT Ruler neu speichern
output_dir_ruler = Path("custom_spacy_model_with_ruler")
output_dir_ruler.mkdir(exist_ok=True)
nlp2.to_disk(output_dir_ruler)
print(f"✅ Modell mit EntityRuler gespeichert unter: {output_dir_ruler.resolve()}")

✅ Modell mit EntityRuler gespeichert unter: /Users/timonmartens/Library/CloudStorage/OneDrive-Persönlich/Desktop/Veranstaltungen/Data Analytics in Applications/daia-eon/notebooks/3_model_training_and_testing/spacy_pipeline/custom_spacy_model_with_ruler


In [51]:
from spacy.training import Example
from spacy.scorer import Scorer
import pandas as pd
from collections import defaultdict

def evaluate_model_with_counts(nlp, examples):
    scorer = Scorer()
    example_list = []

    # Vorbereitung: TP/FP/FN zählen
    counts = defaultdict(lambda: {"tp": 0, "fp": 0, "fn": 0})

    for text, ann in examples:
        doc = nlp(text)
        example = Example.from_dict(doc, ann)
        pred_ents = {(ent.start_char, ent.end_char, ent.label_) for ent in example.predicted.ents}
        true_ents = {(ent.start_char, ent.end_char, ent.label_) for ent in example.reference.ents}

        for ent in pred_ents:
            if ent in true_ents:
                counts[ent[2]]["tp"] += 1
            else:
                counts[ent[2]]["fp"] += 1
        for ent in true_ents:
            if ent not in pred_ents:
                counts[ent[2]]["fn"] += 1

        example_list.append(example)

    # Evaluation mit spaCy
    scores = scorer.score(example_list)
    results = []

    for label, metrics in scores["ents_per_type"].items():
        tp = counts[label]["tp"]
        fp = counts[label]["fp"]
        fn = counts[label]["fn"]

        results.append({
            "Label": label,
            "Precision (%)": round(metrics["p"] * 100, 2),
            "Recall (%)": round(metrics["r"] * 100, 2),
            "F1-Score (%)": round(metrics["f"] * 100, 2),
            "True Positives": tp,
            "False Positives": fp,
            "False Negatives": fn
        })

    return pd.DataFrame(results)

In [52]:
nlp = spacy.load("custom_spacy_model_doccano_labeling")
nlp2 = spacy.load("custom_spacy_model_with_ruler")

results_nlp = evaluate_model_with_counts(nlp, dev_data)
results_nlp2= evaluate_model_with_counts(nlp2, dev_data)
display(results_nlp)
display(results_nlp2)


,Label,Precision (%),Recall (%),F1-Score (%),True Positives,False Positives,False Negatives
0,VORNAME,93.55,93.55,93.55,29,2,2
1,NACHNAME,85.37,92.11,88.61,35,6,3
2,TELEFONNUMMER,37.50,75.00,50.00,3,5,1
3,VERTRAGSNUMMER,76.47,92.86,83.87,13,4,1
4,STRASSE,88.89,100.00,94.12,8,1,0
5,HAUSNUMMER,88.89,100.00,94.12,8,1,0
6,POSTLEITZAHL,88.89,100.00,94.12,8,1,0
7,WOHNORT,100.00,87.50,93.33,7,0,1
8,FIRMA,0.00,0.00,0.00,0,0,3
9,FAX,0.00,0.00,0.00,0,0,2


,Label,Precision (%),Recall (%),F1-Score (%),True Positives,False Positives,False Negatives
0,VORNAME,93.55,93.55,93.55,29,2,2
1,NACHNAME,85.37,92.11,88.61,35,6,3
2,TELEFONNUMMER,37.50,75.00,50.00,3,5,1
3,VERTRAGSNUMMER,76.47,92.86,83.87,13,4,1
4,STRASSE,88.89,100.00,94.12,8,1,0
5,HAUSNUMMER,88.89,100.00,94.12,8,1,0
6,POSTLEITZAHL,88.89,100.00,94.12,8,1,0
7,WOHNORT,100.00,87.50,93.33,7,0,1
8,FIRMA,0.00,0.00,0.00,0,0,3
9,FAX,0.00,0.00,0.00,0,0,2
